In [159]:
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns; sns.set()
%matplotlib inline

import keras
from keras import objectives
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Multiply, Add
from keras.optimizers import Adam, Nadam
import salty
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from random import shuffle
import pandas as pd
import random

#Keras build
from keras import backend as K
from keras.objectives import binary_crossentropy #objs or losses
from keras.models import Model
from keras.layers import Input, Dense, Lambda, Layer
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

#cation data
cations = pd.read_csv('../data/cations.csv')
cations = cations['smiles_string']
salts = pd.read_csv('../data/salts.csv')
salts = salts['smiles_string']
categories = pd.read_csv('../data/categories.csv')
categories = categories['category']
coldic = pd.read_csv('../data/coldic.csv')
coldic = coldic.to_dict(orient='records')[0]
salt_coldic = pd.read_csv('../data/salt_coldic.csv')
salt_coldic = salt_coldic.to_dict(orient='records')[0]
salt_categories = pd.read_csv('../data/salt_categories.csv')
salt_categories = salt_categories['category']
density_coldic = pd.read_csv('../data/density_coldic.csv')
density_coldic = density_coldic.to_dict(orient='records')[0]
density_categories = pd.read_csv('../data/density_categories.csv')
density_categories = density_categories['category']

#supporting functions
import sys
sys.path.insert(0, '../')
from scripts import *

#training array info
smile_max_length = 105
import json
f = open("../data/salt_char_to_index.json","r")
ani_char_to_index = json.loads(f.read())
ani_char_set = set(ani_char_to_index.keys())
ani_char_list = list(ani_char_to_index.keys())
ani_chars_in_dict = len(ani_char_list)
ani_index_to_char = dict((i, c) for i, c in enumerate(ani_char_list))

#training array info
import json
f = open("../data/gdb_char_to_index.json","r")
cat_char_to_index = json.loads(f.read())
cat_char_set = set(cat_char_to_index.keys())
cat_char_list = list(cat_char_to_index.keys())
cat_chars_in_dict = len(cat_char_list)
cat_index_to_char = dict((i, c) for i, c in enumerate(cat_char_list))

In [160]:
anionvae = MoleculeVAE()
anionvae.create(char_set, max_length=62, 
                weights_file='../models/1mil_GDB17_62smi_saltchar_500k_mix_anion_5.h5')
anionvae.autoencoder.layers

In [161]:
cationvae = MoleculeVAE()
cationvae.create(cat_char_set, max_length=51, 
                 weights_file='../models/1mil_GDB17_500K_mix_500K_cation_5.h5')
cationvae.autoencoder.layers

In [162]:
cationvae.save('cation_vae.h5')
anionvae.save('anion_vae.h5')

In [163]:
saltvae = TwoMoleculeVAE()
saltvae.create(cat_char_set, ani_char_set, cat_weights_file='cation_vae.h5',
               ani_weights_file='anion_vae.h5')

In [170]:
seed = salts[random.randint(0,len(salts))].split('.')[0]
print("seed:\t{}".format(seed))
string = ""

for i in saltvae.cation_autoencoder.predict(one_hot(seed, cat_char_to_index, 
                                               smile_max_length=51))[:1]:
    if len(i.shape) > 2:
        i = i[0] #for qspr chemvae there is an extra dim
    for j in i:
#         print(j.shape)
        index = sample(j, temperature=0.2)
        string += cat_index_to_char[index]
print("sample:\t{}".format(string))

seed:	CCCCCCC[N+](C)(C)CC
sample:	CCCCCCC[N+](C)(C)C)1(NCF54 s#13NN5O#3 H78HnN73CO82S


In [167]:
seed = salts[random.randint(0,len(salts))].split('.')[1]
print("seed:\t{}".format(seed))
string = ""

for i in saltvae.anion_autoencoder.predict(one_hot(seed, ani_char_to_index, 
                                               smile_max_length=62))[:1]:
    if len(i.shape) > 2:
        i = i[0] #for qspr chemvae there is an extra dim
    for j in i:
#         print(j.shape)
        index = sample(j, temperature=0.5)
        string += ani_index_to_char[index]
print("sample:\t{}".format(string))

seed:	c1c(cn[n-]1)[N+](=O)[O-]
sample:	c1c(cn[n-]1)[N+](=O)[O-]                                      


In [155]:
##plot
#import matplotlib.pylab as plt
#import seaborn as sns; sns.set()
#from matplotlib import colors
#from itertools import cycle

#data
import pandas as pd
import numpy as np
from numpy.linalg import norm
import json
import random
import copy

#ML
import keras
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import backend as K
from keras import objectives
from keras.objectives import binary_crossentropy #objs or losses
from keras.layers import Dense, Dropout, Input, Multiply, Add, Lambda
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

#chem
#import salty
#from rdkit import Chem
#from rdkit.Chem.Fingerprints import FingerprintMols
#from rdkit import DataStructs
#from rdkit.Chem import Draw


class TwoMoleculeVAE():

    cation_autoencoder = None
    anion_autoencoder = None
    
    def create(self,
               cat_charset,
               ani_charset,
               cat_max_length = 51,
               cat_weights_file = None,
               ani_max_length = 62,
               ani_weights_file = None,
               latent_rep_size = 292,
               qspr_weights_file = None,
               qspr = False):
        
        ### cation
        max_length = cat_max_length
        charset = cat_charset
        charset_length = len(charset)
        x = Input(shape=(max_length, charset_length))
        _, z = self._buildEncoder(x, latent_rep_size, max_length)
        self.cation_encoder = Model(x, z)
#         for layer in self.cation_encoder.layers:
#             layer.name = 'cation_' + layer.name
            
        encoded_input = Input(shape=(latent_rep_size,))
        self.cation_decoder = Model(
            encoded_input,
            self._buildDecoder(
                encoded_input,
                latent_rep_size,
                max_length,
                charset_length
            )
        )
#         for layer in self.cation_decoder.layers:
#             layer.name = 'cation_' + layer.name

        x1 = Input(shape=(max_length, charset_length))
        vae_loss, z1 = self._buildEncoder(x1, latent_rep_size, max_length)
        if qspr:
            self.cation_autoencoder = Model(
                x1,
                self._buildDecoderQSPR(
                    z1,
                    latent_rep_size,
                    max_length,
                    charset_length
                )
            )
#         for layer in self.cation_autoencoder.layers:
#             layer.name = 'cation_' + layer.name
        else:
            self.cation_autoencoder = Model(
                x1,
                self._buildDecoder(
                    z1,
                    latent_rep_size,
                    max_length,
                    charset_length
                )
            )
#         for layer in self.cation_autoencoder.layers:
#             layer.name = 'cation_' + layer.name
            
        ### anion
        max_length = ani_max_length
        charset = ani_charset
        charset_length = len(ani_charset)
        x = Input(shape=(max_length, charset_length))
        _, z = self._buildEncoder(x, latent_rep_size, max_length)
        self.anion_encoder = Model(x, z)
#         for layer in self.anion_encoder.layers:
#             layer.name = 'anion_' + layer.name
            
        encoded_input = Input(shape=(latent_rep_size,))
        self.anion_decoder = Model(
            encoded_input,
            self._buildDecoder(
                encoded_input,
                latent_rep_size,
                max_length,
                charset_length
            )
        )
#         for layer in self.anion_decoder.layers:
#             layer.name = 'anion_' + layer.name

        x1 = Input(shape=(max_length, charset_length))
        vae_loss, z1 = self._buildEncoder(x1, latent_rep_size, max_length)
        if qspr:
            self.anion_autoencoder = Model(
                x1,
                self._buildDecoderQSPR(
                    z1,
                    latent_rep_size,
                    max_length,
                    charset_length
                )
            )
#         for layer in self.anion_autoencoder.layers:
#             layer.name = 'anion_' + layer.name
        else:
            self.anion_autoencoder = Model(
                x1,
                self._buildDecoder(
                    z1,
                    latent_rep_size,
                    max_length,
                    charset_length
                )
            )
#         for layer in self.anion_autoencoder.layers:
#             layer.name = 'anion_' + layer.name
            
        self.qspr = Model(
            x1,
            self._buildQSPR(
                z1,
                latent_rep_size,
                max_length,
                charset_length
            )
        )
        

        if cat_weights_file:
            self.cation_autoencoder.load_weights(cat_weights_file, by_name = True)
            self.cation_encoder.load_weights(cat_weights_file, by_name = True)
            self.cation_decoder.load_weights(cat_weights_file, by_name = True)
        if ani_weights_file:
            self.anion_autoencoder.load_weights(ani_weights_file, by_name = True)
            self.anion_encoder.load_weights(ani_weights_file, by_name = True)
            self.anion_decoder.load_weights(ani_weights_file, by_name = True)
#         if qspr_weights_file:
#             self.qspr.load_weights(weights_file, by_name = True)
        if qspr:
            self.autoencoder.compile(optimizer = 'Adam',
                                     loss = {'decoded_mean': vae_loss, 'qspr': 'mean_squared_error'},
                                     metrics = ['accuracy', 'mse'])
        else:
            self.cation_autoencoder.compile(optimizer = 'Adam',
                                     loss = {'decoded_mean': vae_loss},
                                     metrics = ['accuracy'])
            self.anion_autoencoder.compile(optimizer = 'Adam',
                                     loss = {'decoded_mean': vae_loss},
                                     metrics = ['accuracy'])
    def _buildEncoder(self, x, latent_rep_size, max_length, epsilon_std = 0.01):
        h = Convolution1D(9, 9, activation = 'relu', name='conv_1')(x)
        h = Convolution1D(9, 9, activation = 'relu', name='conv_2')(h)
        h = Convolution1D(10, 11, activation = 'relu', name='conv_3')(h)
        h = Flatten(name='flatten_1')(h)
        h = Dense(435, activation = 'relu', name='dense_1')(h)

        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., stddev = epsilon_std)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(latent_rep_size, name='z_mean', activation = 'linear')(h)
        z_log_var = Dense(latent_rep_size, name='z_log_var', activation = 'linear')(h)

        def vae_loss(x, x_decoded_mean):
            x = K.flatten(x)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = -1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))

    def _buildDecoderQSPR(self, cat_z, ani_z, latent_rep_size, max_length, charset_length):

        h = Dense(latent_rep_size, name='cation_latent_input', activation = 'relu')(cat_z)
        h = RepeatVector(max_length, name='cation_repeat_vector')(h)
        h = GRU(501, return_sequences = True, name='cation_gru_1')(h)
        h = GRU(501, return_sequences = True, name='cation_gru_2')(h)
        h = GRU(501, return_sequences = True, name='cation_gru_3')(h)
        cat_smiles_decoded = TimeDistributed(Dense(charset_length, activation='softmax'), name='cation_decoded_mean')(h)
        
        h = Dense(latent_rep_size, name='anion_latent_input', activation = 'relu')(ani_z)
        h = RepeatVector(max_length, name='anion_repeat_vector')(h)
        h = GRU(501, return_sequences = True, name='anion_gru_1')(h)
        h = GRU(501, return_sequences = True, name='anion_gru_2')(h)
        h = GRU(501, return_sequences = True, name='anion_gru_3')(h)
        ani_smiles_decoded = TimeDistributed(Dense(charset_length, activation='softmax'), name='anion_decoded_mean')(h)

        h = Dense(latent_rep_size, name='qspr_input', activation='relu')([cat_z, ani_z])
        h = Dense(100, activation='relu', name='hl_1')(h)
        h = Dropout(0.5)(h)
        smiles_qspr = Dense(1, activation='linear', name='qspr')(h)

        return smiles_decoded, smiles_qspr

    def _buildDecoder(self, z, latent_rep_size, max_length, charset_length):

        h = Dense(latent_rep_size, name='latent_input', activation = 'relu')(z)
        h = RepeatVector(max_length, name='repeat_vector')(h)
        h = GRU(501, return_sequences = True, name='gru_1')(h)
        h = GRU(501, return_sequences = True, name='gru_2')(h)
        h = GRU(501, return_sequences = True, name='gru_3')(h)
        smiles_decoded = TimeDistributed(Dense(charset_length, activation='softmax'), name='decoded_mean')(h)

        return smiles_decoded
    
    def _buildQSPR(self, z, latent_rep_size, max_length, charset_length):
        h = Dense(latent_rep_size, name='latent_input', activation='relu')(z)
        h = Dense(100, activation='relu', name='hl_1')(h)
        h = Dropout(0.5)(h)
        return Dense(1, activation='linear', name='qspr')(h)

    def save(self, filename):
        self.autoencoder.save_weights(filename)
    
    def load(self, charset, weights_file, latent_rep_size = 292):
        self.create(charset, weights_file = weights_file, latent_rep_size = latent_rep_size)